<a href="https://colab.research.google.com/github/meishihna/GITHUB-COLAB/blob/main/BACKTESTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **工具**

---



In [2]:
!pip install numpy
!pip install pandas_ta
!pip install ta-lib-bin > log.txt
!pip install Ta-Lib
!pip install backtrader
!pip install mplfinance
!pip install backtesting
!pip install FinMind

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=91e143d2c90efec45e440f0c77ac66c3f70bafdd6728e990d47ba5be94ac7c80
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for Ta-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for Ta-Lib
Failed to build Ta-Lib
ERROR: 

In [3]:
import talib
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas_ta as ta
from datetime import datetime, timedelta
from matplotlib.font_manager import fontManager
from io import BytesIO

In [4]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [5]:
# fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
# mpl.rc('font', family='Taipei Sans TC Beta')

# **SMA**

---







In [6]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
from backtesting.lib import SignalStrategy
from talib import abstract
from backtesting.test import SMA
stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2008-01-01', end_date='2025-11-06')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})

class SmaCross(Strategy):
    n1 = 5
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()


bt = Backtest(df1, SmaCross,
              cash=10000, commission=.004,
              exclusive_orders=True)

output = bt.run()
print(output)
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


請輸入要查詢的股票代碼:3035


2024-01-11 12:47:31.437 | INFO     | FinMind.data.finmind_api:get_data:123 - download TaiwanStockPrice, data_id: 3035


Start                     2008-01-02 00:00:00
End                       2024-01-11 00:00:00
Duration                   5853 days 00:00:00
Exposure Time [%]                   55.397511
Equity Final [$]                   89663.4954
Equity Peak [$]                   130234.2394
Return [%]                         796.634954
Buy & Hold Return [%]              480.357143
Return (Ann.) [%]                   15.073453
Volatility (Ann.) [%]                 43.0809
Sharpe Ratio                         0.349887
Sortino Ratio                        0.673882
Calmar Ratio                         0.255642
Max. Drawdown [%]                  -58.963177
Avg. Drawdown [%]                  -10.894519
Max. Drawdown Duration     2876 days 00:00:00
Avg. Drawdown Duration      137 days 00:00:00
# Trades                                  118
Win Rate [%]                        36.440678
Best Trade [%]                      89.658035
Worst Trade [%]                    -17.371331
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1325', ...)

In [7]:
output['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,170,32,49,58.7340,59.4,113.2200,0.011339,2008-02-25,2008-03-20,24 days
1,157,55,67,64.2560,64.5,38.3080,0.003797,2008-03-28,2008-04-16,19 days
2,145,73,78,69.5772,64.9,-678.1940,-0.067223,2008-04-24,2008-05-02,8 days
3,136,81,95,69.2760,61.7,-1030.3360,-0.109360,2008-05-07,2008-05-27,20 days
4,173,131,146,48.7944,49.6,139.3688,0.016510,2008-07-16,2008-08-07,22 days
...,...,...,...,...,...,...,...,...,...,...
113,343,3776,3827,169.6760,286.5,40070.6320,0.688512,2023-05-23,2023-08-07,76 days
114,273,3836,3857,358.9300,335.5,-6396.3900,-0.065277,2023-08-18,2023-09-18,31 days
115,267,3873,3884,343.3680,310.5,-8775.7560,-0.095722,2023-10-13,2023-10-30,17 days
116,251,3894,3923,330.8180,354.0,5818.6820,0.070075,2023-11-13,2023-12-22,39 days


**最佳化回測交易**

In [8]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
from backtesting.lib import SignalStrategy
from talib import abstract
from backtesting.test import SMA

stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2008-01-01', end_date='2025-01-06')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})

class SmaCross(Strategy): #交易策略命名為SmaClass，使用backtesting.py的Strategy功能
    n1 = 5 #設定第一條均線日數為5日(周線)
    n2 = 20 #設定第二條均線日數為20日(月線)，這邊的日數可自由調整

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1) #定義第一條均線為sma1，使用backtesting.py的SMA功能算繪
        self.sma2 = self.I(SMA, self.data.Close, self.n2) #定義第二條均線為sma2，使用backtesting.py的SMA功能算繪

    def next(self):
        if crossover(self.sma1, self.sma2): #如果周線衝上月線，表示近期是上漲的，則買入
            self.buy()
        elif crossover(self.sma2, self.sma1): #如果周線再與月線交叉，表示開始下跌了，則賣出
            self.position.close()



test = Backtest(df1, SmaCross,
              cash=10000, commission=.004,
              exclusive_orders=True)


result = test.run() #原始結果

opt_result = test.optimize(n1=range(5, 60, 5),  #將回測機器加入optimize屬性，定義短均線的周期為5~50(每次加5)，長均線的周期為10~120(每次加5)
                    n2=range(10, 120, 5),
                    maximize='Equity Final [$]',  #最佳化目標為最終資產最大化
                    constraint=lambda p: p.n1 < p.n2)  #限制n1及n2的範圍，只會計算n1小於n2的情況

print("Optimize strategy")
print(opt_result)  #印出opt_result結果

請輸入要查詢的股票代碼:3035


2024-01-11 12:47:47.642 | INFO     | FinMind.data.finmind_api:get_data:123 - download TaiwanStockPrice, data_id: 3035


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Optimize strategy
Start                     2008-01-02 00:00:00
End                       2024-01-11 00:00:00
Duration                   5853 days 00:00:00
Exposure Time [%]                   52.095504
Equity Final [$]                  207696.9072
Equity Peak [$]                   212157.4472
Return [%]                        1976.969072
Buy & Hold Return [%]              480.357143
Return (Ann.) [%]                   21.430066
Volatility (Ann.) [%]               45.016285
Sharpe Ratio                         0.476051
Sortino Ratio                        0.942046
Calmar Ratio                         0.419698
Max. Drawdown [%]                  -51.060714
Avg. Drawdown [%]                  -11.068817
Max. Drawdown Duration     1256 days 00:00:00
Avg. Drawdown Duration      125 days 00:00:00
# Trades                                   36
Win Rate [%]                        36.111111
Best Trade [%]                     474.376361
Worst Trade [%]                    -20.816733
Avg. Trade [%]  

In [9]:
print(opt_result['_strategy'])
opt_result['_trades']

SmaCross(n1=5,n2=90)


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,319,270,406,31.3248,52.00,6595.3888,0.660027,2009-02-10,2009-08-24,195 days
1,296,421,455,55.9228,53.50,-717.1488,-0.043324,2009-09-14,2009-10-30,46 days
2,274,471,524,57.8304,56.50,-364.5296,-0.023005,2009-11-23,2010-02-05,74 days
3,256,543,546,60.4408,59.70,-189.6448,-0.012257,2010-03-12,2010-03-17,5 days
4,279,639,642,54.8184,54.10,-200.4336,-0.013105,2010-07-28,2010-08-02,5 days
5,279,646,711,54.1156,55.00,246.7476,0.016343,2010-08-06,2010-11-08,94 days
6,256,728,750,60.0392,57.30,-701.2352,-0.045624,2010-12-01,2010-12-31,30 days
7,244,756,777,60.0392,58.00,-497.5648,-0.033964,2011-01-10,2011-02-16,37 days
8,485,994,1075,29.2164,39.95,5205.7960,0.367383,2011-12-27,2012-04-30,125 days
9,459,1081,1082,42.1680,42.00,-77.1120,-0.003984,2012-05-09,2012-05-10,1 days


# **KDJ**

---



In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
import pandas as pd
import talib
from talib import abstract

stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2013-01-01', end_date='2025-12-31')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})
## 取得 KD 值
df_kd = abstract.STOCH(df2,fastk_period=9, slowk_period=3,slowd_period=3)
## 合併資料
df1 = pd.merge(df1, df_kd, on="Date")

def I_bypass(data): # bypass data in Strategy
    return data

## KD 策略
class KdCross(Strategy):
    lower_bound = 20
    upper_bound = 80
    # sl_ratio = 99     # stop loss ratio, 99 means 1% loss
    def init(self):
        self.k = self.I(I_bypass, self.data.slowk) #K
        self.d = self.I(I_bypass, self.data.slowd) #D

    def next(self):
        if crossover(self.k, self.d) and self.k<self.lower_bound and self.d<self.lower_bound and not self.position: #long position
            self.buy()
        elif crossover(self.d, self.k) and self.k>self.upper_bound and self.d>self.upper_bound:
            if self.position and self.position.is_long:
                self.position.close()
bt = Backtest(df1, KdCross, cash=10000, commission=.004) ## 交易成本
output = bt.run()
print(output)
bt.plot()

In [ ]:
output['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,988,20,74,10.12020,12.95,2795.84240,0.279619,2013-01-30,2013-04-29,89 days
1,974,108,184,13.12620,12.90,-220.31880,-0.017233,2013-06-18,2013-10-04,108 days
2,972,203,245,12.92580,13.05,120.72240,0.009609,2013-11-01,2013-12-31,60 days
3,974,275,409,13.02600,9.92,-3025.24400,-0.238446,2014-02-20,2014-09-03,195 days
4,1035,419,434,9.33864,9.05,-298.74240,-0.030908,2014-09-18,2014-10-09,21 days
5,1279,442,452,7.32462,8.43,1413.78102,0.150913,2014-10-22,2014-11-05,14 days
6,1226,506,534,8.79756,8.50,-364.80856,-0.033823,2015-01-21,2015-03-11,49 days
7,1223,538,555,8.51700,9.32,982.06900,0.094282,2015-03-17,2015-04-13,27 days
8,1302,564,613,8.75748,10.00,1617.76104,0.141881,2015-04-24,2015-07-06,73 days
9,1772,629,733,7.34466,7.00,-610.73752,-0.046927,2015-07-29,2015-12-25,149 days


**最佳化回測交易**

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
import pandas as pd
import talib
from talib import abstract

stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2008-01-01', end_date='2025-12-25')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})
## 取得 KD 值
df_kd = abstract.STOCH(df2,fastk_period=9, slowk_period=3,slowd_period=3)
## 合併資料
df1 = pd.merge(df1, df_kd, on="Date")

def I_bypass(data): # bypass data in Strategy
    return data

## KD 策略
class KdCross(Strategy):
    lower_bound = 20
    upper_bound = 80
    # sl_ratio = 99     # stop loss ratio, 99 means 1% loss
    def init(self):
        self.k = self.I(I_bypass, self.data.slowk) #K
        self.d = self.I(I_bypass, self.data.slowd) #D

    def next(self):
        if crossover(self.k, self.d) and self.k<self.lower_bound and self.d<self.lower_bound and not self.position: #long position
            self.buy()
        elif crossover(self.d, self.k) and self.k>self.upper_bound :# and self.d>self.upper_bound:
            if self.position and self.position.is_long:
                self.position.close()

bt = Backtest(df1, KdCross, cash=10000, commission=.004) ## 交易成本

opt_result = bt.optimize(lower_bound=range(10, 40, 3),  #將回測機器加入optimize屬性，定義短均線的周期、長均線的周期
                    upper_bound=range(60, 90, 3),
                    maximize='Equity Final [$]', #最佳化目標為最終資產最大化
                    # return_heatmap = True,
                    )
print("Optimize strategy")
print(opt_result)  #印出opt_result結果

請輸入要查詢的股票代碼:2405


2024-01-03 11:47:40.577 | INFO     | FinMind.data.finmind_api:get_data:123 - download TaiwanStockPrice, data_id: 2405


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Optimize strategy
Start                     2008-01-02 00:00:00
End                       2023-12-25 00:00:00
Duration                   5836 days 00:00:00
Exposure Time [%]                   82.693774
Equity Final [$]                  67101.85664
Equity Peak [$]                   72309.65664
Return [%]                         571.018566
Buy & Hold Return [%]               61.926606
Return (Ann.) [%]                   12.965187
Volatility (Ann.) [%]               46.141318
Sharpe Ratio                         0.280989
Sortino Ratio                        0.512643
Calmar Ratio                         0.188654
Max. Drawdown [%]                   -68.72465
Avg. Drawdown [%]                  -12.743979
Max. Drawdown Duration     2852 days 00:00:00
Avg. Drawdown Duration      237 days 00:00:00
# Trades                                   41
Win Rate [%]                         60.97561
Best Trade [%]                      93.809146
Worst Trade [%]                    -36.869065
Avg. Trade [%]  

In [ ]:
print(opt_result['_strategy'])
opt_result['_trades']

KdCross(lower_bound=28,upper_bound=84)


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,986,18,31,10.14040,12.60,2425.16560,0.242555,2008-01-28,2008-02-22,25 days
1,944,53,63,13.15240,15.05,1791.33440,0.144278,2008-03-26,2008-04-10,15 days
2,931,88,92,15.26080,16.45,1107.14520,0.077925,2008-05-16,2008-05-22,6 days
3,997,100,151,15.36120,13.15,-2204.56640,-0.143947,2008-06-03,2008-08-14,72 days
4,1232,169,209,10.64240,7.36,-4043.91680,-0.308427,2008-09-09,2008-11-06,58 days
5,1386,219,316,6.54608,9.00,3401.13312,0.374869,2008-11-20,2009-04-15,146 days
6,1565,323,333,7.97176,15.45,11703.44560,0.938091,2009-04-24,2009-05-11,17 days
7,1433,357,368,16.86720,19.30,3486.20240,0.144233,2009-06-15,2009-06-30,15 days
8,1485,396,439,18.62420,27.80,13626.06300,0.492682,2009-08-10,2009-10-08,59 days
9,1692,450,466,24.39720,26.00,2711.93760,0.065696,2009-10-23,2009-11-16,24 days


# **RSI**

---



In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
from backtesting.lib import SignalStrategy
from talib import abstract
from backtesting.test import SMA
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply

stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2013-01-01', end_date='2025-12-25')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70

    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)

        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)

        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)


    def next(self):
        price = self.data.Close[-1]

        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):

            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)

        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()


bt = Backtest(df1, System,
              cash=10000, commission=.004,
              exclusive_orders=True)

stats= bt.run()
print(stats)
bt.plot()

In [ ]:
stats['_trades']

**最佳化回測交易**

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
from backtesting.lib import SignalStrategy
from talib import abstract
from backtesting.test import SMA
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply

stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2008-01-01', end_date='2025-12-25')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70

    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)

        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)

        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)


    def next(self):
        price = self.data.Close[-1]

        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):

            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)

        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()


bt = Backtest(df1, System,
              cash=10000, commission=.004,
              exclusive_orders=True)

opt_result = bt.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10),maximize='Equity Final [$]')
print("Optimize strategy")
print(opt_result)  #印出opt_result結果

請輸入要查詢的股票代碼:2405


2024-01-03 06:17:54.964 | INFO     | FinMind.data.finmind_api:get_data:123 - download TaiwanStockPrice, data_id: 2405


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Optimize strategy
Start                     2008-01-02 00:00:00
End                       2023-12-25 00:00:00
Duration                   5836 days 00:00:00
Exposure Time [%]                   10.165184
Equity Final [$]                  15675.06764
Equity Peak [$]                   27376.59112
Return [%]                          56.750676
Buy & Hold Return [%]               61.926606
Return (Ann.) [%]                     2.92037
Volatility (Ann.) [%]               19.253421
Sharpe Ratio                         0.151681
Sortino Ratio                        0.253071
Calmar Ratio                         0.059787
Max. Drawdown [%]                   -48.84659
Avg. Drawdown [%]                  -19.372204
Max. Drawdown Duration     2846 days 00:00:00
Avg. Drawdown Duration      806 days 00:00:00
# Trades                                   41
Win Rate [%]                        31.707317
Best Trade [%]                     114.374291
Worst Trade [%]                    -10.657371
Avg. Trade [%]  

In [ ]:
print(opt_result['_strategy'])
opt_result['_trades']

System(d_rsi=10,w_rsi=10,level=30)


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,650,111,112,15.36120,14.260,-715.78000,-0.071687,2008-06-18,2008-06-19,1 days
1,1020,324,353,9.09624,19.500,10611.83520,1.143743,2009-04-27,2009-06-09,43 days
2,1042,369,375,19.07600,19.700,650.20800,0.032711,2009-07-01,2009-07-09,8 days
3,947,379,387,21.68640,20.200,-1407.62080,-0.068541,2009-07-15,2009-07-27,12 days
4,862,418,428,22.18840,20.332,-1600.21680,-0.083665,2009-09-09,2009-09-23,14 days
5,779,430,446,22.48960,24.800,1799.80160,0.102732,2009-09-25,2009-10-19,24 days
6,740,466,485,26.10400,27.350,922.04000,0.047732,2009-11-16,2009-12-11,25 days
7,710,487,506,28.51360,30.100,1126.34400,0.055637,2009-12-15,2010-01-12,28 days
8,1972,1157,1160,10.84320,10.250,-1169.79040,-0.054707,2012-08-24,2012-08-29,5 days
9,1873,1177,1188,10.79300,10.800,13.11100,0.000649,2012-09-21,2012-10-08,17 days


# **MACD**

---














In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
import pandas as pd
import numpy as np
import talib as ta
from talib import abstract,EMA
from backtesting.test import SMA
stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2008-01-01', end_date='2025-12-25')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})
## 取得 KD 值
df_kd = abstract.STOCH(df2,fastk_period=9, slowk_period=3,slowd_period=3)
## 合併資料
df1 = pd.merge(df1, df_kd, on="Date")

# 定義 MACD 策略
def MACD(close, n1, n2, ns):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=ns)
    return macd, macdsignal

class MACDCross(Strategy):
    n1 = 12 #短期EMA期間
    n2 = 26 #長期EMA期間
    ns = 9 #（MACDのSMA）期間

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, self.data.Close, self.n1, self.n2, self.ns)

    def next(self):
        if crossover(self.macd, self.macdsignal):
            self.buy()
        elif crossover(self.macdsignal, self.macd):
            self.position.close()

bt = Backtest(
    df1,
    MACDCross,
    cash=10000,  # 初始資金
    commission=0.00495,  # 交易手續費
    margin=1.0,  # 槓桿倍數的倒數（0.5表示2倍槓桿）
    trade_on_close=True,  # True：以當前收盤價進行交易，False：以下一個時間點的開盤價進行交易
    exclusive_orders=True  # 自動關閉現有持倉（開倉）
)


output = bt.run()
print(output)
bt.plot()

In [ ]:
output['_trades']

**最佳化回測交易**

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
import pandas as pd
import numpy as np
import talib as ta
from talib import abstract,EMA
from backtesting.test import SMA
stock_id = input("請輸入要查詢的股票代碼:")
## 取得資料
dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id, start_date='2008-01-01', end_date='2025-12-31')
## 整理資料格式
df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})
## 取得 KD 值
df_kd = abstract.STOCH(df2,fastk_period=9, slowk_period=3,slowd_period=3)
## 合併資料
df1 = pd.merge(df1, df_kd, on="Date")

# 定義 MACD 策略
def MACD(close, n1, n2, ns):
    dif, macd, macdhist = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=ns)
    return dif, macd

class MACDCross(Strategy):
    n1 = 12 #短期EMA期間
    n2 = 26 #長期EMA期間
    ns = 9 #（MACDのSMA）期間

    def init(self):
        self.dif, self.macd = self.I(MACD, self.data.Close, self.n1, self.n2, self.ns)

    def next(self):
        if crossover(self.dif, self.macd) and self.dif < 0.5:
            self.buy()
        elif crossover(self.macd, self.dif):
            self.position.close()

bt = Backtest(
    df1,
    MACDCross,
    cash=10000,  # 初始資金
    commission=0.004,  # 交易手續費
    margin=1.0,  # 槓桿倍數的倒數（0.5表示2倍槓桿）
    trade_on_close=True,  # True：以當前收盤價進行交易，False：以下一個時間點的開盤價進行交易
    exclusive_orders=True  # 自動關閉現有持倉（開倉）
)

#最適化
output2=bt.optimize(n1=range(10, 50, 2),
          n2=range(20, 250, 5),
          ns=range(5, 50, 5),
          maximize='Equity Final [$]',
          method='grid')
print(output2)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
2024-01-03 11:08:26.821 | INFO     | FinMind.data.finmind_api:get_data:123 - download TaiwanStockPrice, data_id: 2405
/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 8280 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/28 [00:00<?, ?it/s]

Start                     2008-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   5840 days 00:00:00
Exposure Time [%]                   40.035542
Equity Final [$]                   40568.6574
Equity Peak [$]                    45846.6574
Return [%]                         305.686574
Buy & Hold Return [%]               66.513761
Return (Ann.) [%]                    9.372812
Volatility (Ann.) [%]               29.884896
Sharpe Ratio                          0.31363
Sortino Ratio                        0.577245
Calmar Ratio                         0.144716
Max. Drawdown [%]                  -64.767151
Avg. Drawdown [%]                  -13.109363
Max. Drawdown Duration     2985 days 00:00:00
Avg. Drawdown Duration      271 days 00:00:00
# Trades                                   68
Win Rate [%]                        33.823529
Best Trade [%]                     171.154645
Worst Trade [%]                     -16.46318
Avg. Trade [%]                    

In [ ]:
print(output2['_strategy'])
output2['_trades']

MACDCross(n1=12,n2=245,ns=10)


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1527,265,355,6.54608,17.75,17108.38584,1.711546,2009-02-03,2009-06-11,128 days
1,1122,603,620,24.14620,24.25,116.46360,0.004299,2010-06-07,2010-07-01,24 days
2,1069,649,658,25.45140,24.30,-1230.84660,-0.045239,2010-08-11,2010-08-24,13 days
3,1041,670,692,24.94940,25.00,52.67460,0.002028,2010-09-09,2010-10-12,33 days
4,1150,730,741,22.64020,21.00,-1886.23000,-0.072446,2010-12-03,2010-12-20,17 days
...,...,...,...,...,...,...,...,...,...,...
63,3212,3659,3685,12.14840,12.50,1129.33920,0.028942,2022-11-04,2022-12-12,38 days
64,3319,3712,3740,12.09820,12.30,669.77420,0.016680,2023-01-31,2023-03-14,42 days
65,3189,3758,3769,12.80100,12.35,-1438.23900,-0.035232,2023-04-12,2023-04-27,15 days
66,3052,3771,3776,12.90140,12.35,-1682.87280,-0.042740,2023-05-02,2023-05-09,7 days
